In [1]:
import pandas as pd

## 行转列

In [2]:
df = pd.read_csv("./row2column.csv")
# df.head(5)

# df.set_index(["Country","TIME"])["Value"].unstack().rename_axis(columns=None).head(5)

df.pivot("Country","TIME","Value").rename_axis(columns=None).head(5)

,2017,2018,2019,2020
Country,,,,
ARG,18.388876,39.957542,NaN,NaN
AUS,51.989063,51.393173,52.478458,54.614105
AUT,40.324657,40.458271,41.610401,41.419125
BEL,45.725086,47.397144,47.282982,48.508053
BRA,19.613668,21.314569,21.602325,23.531902


## 一行生成多行

In [15]:
df = pd.read_excel("./oneRow2many.xlsx")

# df = df.set_index(["声优","生日"])["作品"].str.split(",",expand=True)
# df.stack().reset_index(drop=True,level=-1).reset_index().rename(columns={0: "配音角色"})

df["作品"] = df["作品"].str.split(",")
# df["作品"] = df["作品"].map(lambda x: x.split(","))
df.explode("作品")


,声优,生日,作品
0,关智一,1972-09-08,吉尔伽美什
0,关智一,1972-09-08,罗布·路奇
0,关智一,1972-09-08,桥田至
1,松冈祯丞,1986-09-17,贝尔·克朗尼
1,松冈祯丞,1986-09-17,桐人
1,松冈祯丞,1986-09-17,神田空太
1,松冈祯丞,1986-09-17,安艺伦也
2,雨宫天,1993-08-28,赤瞳
2,雨宫天,1993-08-28,雾嶋董香
2,雨宫天,1993-08-28,米娅


## 字典拆分成多列

In [4]:
df = pd.read_excel("./dict2columns.xlsx")
df["info"] = df["info"].apply(eval) # 先把数据格式转化为dict
tmp = df["info"].apply(pd.Series)
# 因为我们这里的值是一个字典, 而Series接收一个字典的话, 那么字典的key就是索引, value就是值
# 在扩展成DataFrame的时候同样会考虑到字典中所有的key, 有多少个不重复的key就会生成多少个列
# 如果该行没有对应的值则使用NaN填充
df[tmp.columns] = tmp # 将tmp添加到df中
df = df.drop(columns="info")
df


,id,声优,生日,作品,外号
0,0,关智一,1972-09-08,"吉尔伽美什,罗布·路奇,桥田至",NaN
1,1,松冈祯丞,1986-09-17,"贝尔·克朗尼,桐人,神田空太,安艺伦也",后宫王
2,2,雨宫天,1993-08-28,"赤瞳,雾嶋董香,米娅,阿克娅",NaN


## 列转行

In [21]:
df = pd.read_excel("./column2row.xlsx")

# 内置melt
pd.melt(df,id_vars=["姓名","水果"],value_vars=["星期一","星期二","星期三"],var_name="星期几?",value_name="销量")


# 用stack来实现一个melt
def my_melt(frame: pd.DataFrame,
            id_vars: list,
            value_vars: list,
            var_name: str,
            value_name: str):
    # 先将id_vars和value_vars指定的字段筛选出来
    frame = frame[id_vars + value_vars]

    # 将id_vars指定的字段设置为索引
    frame = frame.set_index(id_vars)
    print(">>>筛选字段、设置索引之后对应的DataFrame:\n", frame)

    # 调用frame的stack方法, 会得到一个具有多级索引的Series
    # frame的列就是生成的Series的最后一级索引
    s = frame.stack()
    print(">>>得到的具有多级索引的Series:\n", s)

    # 直接对该Series进行reset_index即可, 会得到DataFrame, 将索引变成列
    frame = s.reset_index()
    print(">>>具有多级索引的Series执行reset_index:\n", frame)

    # 重命名
    frame.columns = id_vars + [var_name, value_name]
    print(">>>最终返回的DataFrame:")
    return frame

,姓名,水果,level_2,0
0,古明地觉,草莓,星期一,70斤
1,古明地觉,草莓,星期二,72斤
2,古明地觉,草莓,星期三,60斤
3,雾雨魔理沙,樱桃,星期一,61斤
4,雾雨魔理沙,樱桃,星期二,60斤
5,雾雨魔理沙,樱桃,星期三,81斤
6,琪露诺,西瓜,星期一,103斤
7,琪露诺,西瓜,星期二,116斤
8,琪露诺,西瓜,星期三,153斤


## 列转行(键值列)

In [50]:
df = pd.read_excel("./column2row_advanced.xlsx")

# df["技能"] = df[["技能1","技能2","技能3"]].agg(tuple,axis=1)
# df["活动地点"] = df[["活动地点1","活动地点2","活动地点3"]].agg(tuple,axis=1)
# df = df.filter(regex=r"(?<!\d)$") # 筛选出不存在数字的列
# df["技能-活动地点"] = df["技能"].combine(df["活动地点"],func=lambda x,y:list(zip(x,y)))
# df = df.drop(columns=["技能","活动地点"])
# df = df.explode("技能-活动地点")
# df[["技能","活动地点"]] = df["技能-活动地点"].apply(pd.Series)
# df = df.drop(columns=["技能-活动地点"])

df = pd.lreshape(df,{"技能":["技能1","技能2","技能3"],"活动地点":["活动地点1","活动地点2","活动地点3"]}) # 注意不能有空值，否则会被舍弃.要保留空值要用dropna=False
df

,姓名,年龄,技能,活动地点
0,麦克雷,37,闪光弹,66号公路
1,源氏,35,闪,尼泊尔
2,士兵76,55,螺旋飞弹,多拉多
3,麦克雷,37,战术翻滚,多拉多
4,源氏,35,影,花村
5,士兵76,55,生物立场,伊里奥斯
6,麦克雷,37,神射手,漓江塔
7,源氏,35,斩,直布罗陀
8,士兵76,55,战术目镜,努巴尼
